In [3]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload all

from utils import read_images
from datasets import SimpleSeriesSelectionDataset
from models import TemporalResNet

torch.set_grad_enabled(False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
model = TemporalResNet()
model.load_state_dict(torch.load('series_selection.pth'))
model.eval()

images = read_images(ids="test")
test_dataset = SimpleSeriesSelectionDataset(images, split="test")

RuntimeError: Error(s) in loading state_dict for TemporalResNet:
	Missing key(s) in state_dict: "base_model.conv1.bias". 
	size mismatch for fc.weight: copying a param with shape torch.Size([1, 512]) from checkpoint, the shape in current model is torch.Size([2, 512]).
	size mismatch for fc.bias: copying a param with shape torch.Size([1]) from checkpoint, the shape in current model is torch.Size([2]).

In [34]:
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

all_preds = []
all_labels = []

for images, labels in test_loader:
    preds = model(images)
    preds = torch.sigmoid(preds)[:, 1]
    all_preds.extend(preds.cpu().numpy())
    all_labels.extend(labels.cpu().numpy())

In [36]:
test_ids = [idx[0] for idx in test_dataset.index]
test_results = pd.DataFrame({
    'pred_proba': all_preds,
    'label': all_labels
}, index=test_ids)
test_results["pred"] = (test_results["pred_proba"] > 0.5).astype(int)
test_results["correct"] = (test_results["pred"] == test_results["label"])

test_results = test_results[~test_results.index.duplicated(keep=False)]
test_results

,pred_proba,label,pred,correct
131aedfhs6pnf1fvtvp49mfffzgvygi422,0.550009,1,1,True
13c2ur549vohc0jat2ewk4eyyh1,0.535090,1,1,True
131aedfhs6pnf1fvtvp49mmcakxtn8la22,0.574957,1,1,True
131aedfhs6pnf1fvtvp49mj8lu0x093r22,0.542112,1,1,True
131aedfhs6pnf1fvtvp49mn4mfl8nw8t22,0.551948,1,1,True
...,...,...,...,...
131aedfhs6pnf1fvtvp49mkefqvzq2ak22,0.551723,0,1,False
131aedfhs6pnf1fvtvp49mha299w9vi022,0.588702,0,1,False
131aedfhs6pnf1fvtvp49jqsoomstmi322,0.529795,0,1,False
131aedfhs6pnf1fvtvp49mkd12ier3ua22,0.538426,0,1,False


In [2]:
report = classification_report(test_results["labels"], test_results["pred"])
matrix = confusion_matrix(test_results["labels"], test_results["pred"])

print("Classification Report:")
print(report)
print("Confusion Matrix:")
print(matrix)

NameError: name 'test_results' is not defined

In [46]:
irf = pd.read_csv("data/imagerejectionfeedback.csv")
irf = irf.set_index("IMAGE_ID")

rejection_reasons = [
    "TECHNICALVESSELSNOTVISIBLE",
    "TECHNICALVESSELSOVERLAP",
    "TECHNICALNOCONTRAST",
    "TECHNICALLOWQUALITY",
    "PROCEDURALFEMORALACCESS",
    "PROCEDURALELECTRODESVISIBLE",
    "PROCEDURALCATHETERINAORTA",
    "EXAMINATIONAORTOGRAPHY",
    "EXAMINATIONVENTRICULOGRAPHY",
    "PHYSIOLOGICALCRITICALSTENOSIS",
    "PHYSIOLOGICALCOLLATERALCIRCULATION",
]

irf = irf[rejection_reasons]
    
irf = irf.groupby(irf.index).sum()
irf

,TECHNICALVESSELSNOTVISIBLE,TECHNICALVESSELSOVERLAP,TECHNICALNOCONTRAST,TECHNICALLOWQUALITY,PROCEDURALFEMORALACCESS,PROCEDURALELECTRODESVISIBLE,PROCEDURALCATHETERINAORTA,EXAMINATIONAORTOGRAPHY,EXAMINATIONVENTRICULOGRAPHY,PHYSIOLOGICALCRITICALSTENOSIS,PHYSIOLOGICALCOLLATERALCIRCULATION
IMAGE_ID,,,,,,,,,,,
12aw4ack71831bocuf5j3pz238tcmv77343,1,0,0,0,0,0,0,0,0,0,0
12aw4ack71831bocuf5j3pz23ddfsvb226y,0,0,1,0,0,0,0,0,0,0,0
12aw4ack71831bocuf5j3pz23f7d6vf0362,0,0,1,0,0,0,0,0,0,0,0
12aw4ack71831bocuf5j3pz23jx4qvj87o6,0,0,1,0,0,0,0,0,0,0,0
12aw4ack71831bocuf5j3pz23k1ttvn84cw,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
13c2ur549vohc0jau2o8z32yc01,1,0,0,0,0,0,0,0,0,0,0
13c2ur549vohc0jau2o8z32yem1,1,0,0,0,0,0,0,0,0,0,0
13c2ur549vohc0jau2o8z32yiy1,1,0,0,0,0,0,0,0,0,0,0


In [48]:
df = test_results.join(irf).fillna(0)
for r in rejection_reasons:
    f = df[df[r] == 1]["correct"]
    print(f"{r}: {f.sum()}/{len(f)}")

TECHNICALVESSELSNOTVISIBLE: 0/9
TECHNICALVESSELSOVERLAP: 0/7
TECHNICALNOCONTRAST: 1/6
TECHNICALLOWQUALITY: 0/4
PROCEDURALFEMORALACCESS: 0/0
PROCEDURALELECTRODESVISIBLE: 0/0
PROCEDURALCATHETERINAORTA: 0/0
EXAMINATIONAORTOGRAPHY: 0/1
EXAMINATIONVENTRICULOGRAPHY: 0/0
PHYSIOLOGICALCRITICALSTENOSIS: 0/0
PHYSIOLOGICALCOLLATERALCIRCULATION: 0/0
